In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install d2l==0.17.6

In [ ]:
import os
path = '/content/drive/MyDrive'
os.chdir(path)

!source venv_d2l/bin/activate

path = '/content/drive/MyDrive/d2l-zh'
os.chdir(path)

分类猫和狗的图片
- 使用还不错的相机采集图片（1200万像素）
- RGB图片有3600万元素 Input layer 特征
- 使用100大小的单隐藏层MLP，模型有36亿参数=14GB
- 远多于世界上所有猫和狗的总数（9亿狗，6亿猫）
- 单层就要14GB的内存，还不算做运算

Waldo 两个原则 在图片里找waldo的原则启发了我们CNN的设计
- 平移不变性
- 局部性

卷积CNN 特殊的全连接层
- 全连接层，之前的图片是个矩阵，但我们是把他作为一维的向量，现在我们还是还原为矩阵，因为我们要考虑空间的一些信息，必须用矩阵来存
- 将输入和输出从向量变形为矩阵（宽度，高度）
- 将权重变形为4-D张量(h,w)到(h',w') 输入的高宽到输出的高宽
- V是W的重新索引
- 原则#1 - 平移不变性 x的平移导致h的平移 而V不应该依赖于i,j
- 解决方案：不管ij怎么变，v应该不变，v_i,j,a,b = v_a,b
- 不管ij挪到哪里，我要看的识别器v是不发生变化的，这个就叫做2维卷积，数学上来说叫2维交叉相关：输出的像素等于输入ij对应的那个像素，以他为中心不断做offset的加一点减一点往边上挪的时候，和他V的这一个模是做内积
- 平移不变性使得我们把ij这个维度干掉了，只剩ab这个维度
- 2维卷积就是我的全连接，矩阵乘法，但是我的权重使得他的一些东西是重复的，他不是每一个元素都可以自由变换。当我把一个模型的取值范围做限制的话，我就把模型复杂度降低了，同样的话，就意味着我不用存那么多东西了。
- 原则#2 - 局部性
- 假设我要去算ij的这个输出的话，我会取看以ij为中心的所有地方，但实际上来说，我们不应该去看那么多地方，ij的结果只应该由Xij附近的那些点就行了。
- 当评估hij时，我们不应该用远离xij的参数
- 解决方案：当|a|,|b| > delta时，使得v_a,b = 0. ij的点远离我超过delta的时候，那些地方就不再看了
- 总结：对全连接层使用平移不变性和局部性得到卷积层

刚刚是卷积的操作子，现在来说卷积层是什么。
- 二维交叉相关：input, kernel 卷积核，output
- 二维卷积层
- 输入X: n_h x n_w
- 核W: k_h x k_w
- 偏差b 是一个实数
- 输出Y: (n_h - k_h + 1) x (n_w - k_w + 1) 平移到后面没了，丢掉的就是k_h - 1和k_w - 1
- Y = X * W + b
- W和b是可学习的参数，神经网络来学习出一些核
- 交叉相关和卷积的唯一区别是W_a,b和W_-a,-b, 有个负号。由于对称性，在实际使用中没有区别，因为w是我学的，其实就是上下反一下，左右反一下，实际上没区别。神经网络因为写起来方便，所以没有用数学上的卷积写法，其实是倒过来的。说是说卷积层，实际上实现的交叉相关
- CNN来做文本效果很好
- 总结：卷积层就是将输入和核矩阵进行交叉相关运算，加上偏移后得到输出。核矩阵和偏移是可学习的参数。核矩阵的大小是超参数。核参数是个很小的东西，这就是解决了随着输入的变大，参数不会变得特别大，卷积不会有这个问题。



In [ ]:
# 图像卷积
# 互相关运算 cross-correlation
import torch
from torch import nn
from d2l import torch as d2l

# X是输入，K是核矩阵
def corr2d(X, K):
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            # 小方块拿出来跟核矩阵做点积然后求和
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [ ]:
# 验证上述二维互相关运算的输出
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [ ]:
# 实现二维卷积层
# 用继承nn.Module来实现一个类
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size)) # kernel_size超参数
        self.bias = nn.Parameter(torch.zeros(1)) # 标量

    # 前向运算
    def forward(self, x):
        x和weight做互相关运算
        return corr2d(x, self.weight) + self.bias

In [ ]:
# 卷积层的一个简单应用：检测图像中不同颜色的边缘
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [ ]:
# 假设两个元素没有边的话，输出就是0，如果1变0或0变1，输出要么1要么-1
K = torch.tensor([[1.0, -1.0]])
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [ ]:
# 卷积核K只可以检测垂直边缘
# 转置后就不能检测东西了
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [ ]:
# 学习由X生成Y的卷积核
# 现在是给定X和Y，来学这个K

# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
# 调用了nn的Conv2d函数，第一个1是输入的通道，单个矩阵，黑白图片通道为1，彩色图片通道为3
# 第二个1是输出通道，现在也为1；核我们知道是个1x2的核，来学他；不需要bias
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # 学习率

# 手写了一个训练模型 迭代10轮
# 每一次把X放到conv2d里得到Y_hat，然后计算均方误差作为loss
# 然后把conv2d的梯度设为0，loss求和后算个backward，这儿是梯度下降
# conv2d.weight.data[:] 做inplace操作，-=学习率*梯度
# 每两个batch后print loss
for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 4.095
epoch 4, loss 0.799
epoch 6, loss 0.180
epoch 8, loss 0.049
epoch 10, loss 0.016


In [ ]:
# 所学的卷积核的权重向量
# 跟我们自己构造出来的(1,-1)很接近了
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0019, -0.9783]])

为什么kernel不是越大越好？
- 最终是想看到所有的没错。类似于问为什么全连接层为什么不是隐藏层越大越好。可以做一个很浅很宽的全连接层，但很多时候没有一个深一点的窄一点的全连接层效果好。所以神经网络也是一样。
- 我做一个卷积层层数比较少的1-2层，但是每个核比较大的，不然每一层把核变小一点，然后把他做深一点。最后的视野是一样的，每次看一点看一点。所以核主流用3x3，最多用5x5.


房价竞赛时，损失随迭代次数变化图抖动很厉害：
- 可能是数据多样性比较大，这个抖动是没关系的，只要下降
- 可以把批量大小增大或者学习率给高一点

全连接层最大的问题是权重W的高是取决于输入的宽，当给一个1200万像素的图片的时候，输入的维度就变成1200万的维度了。
- 所以用卷积就没这个问题，不管输入多大，核的大小是固定的。
- 实际应用中，我们不会把1200万丢进去，而是200x200的图片，但是全连接层不是那么好用。